# Attribution for recommender system model

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import scipy.sparse as sp
from torch.utils.data import DataLoader, Dataset

In [ ]:
# download dataset
url = "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
!wget -q {url} -O ml-1m.zip
!unzip -q ml-1m.zip

In [17]:
data_path = "ml-1m/ratings.dat"
movies_path = "ml-1m/movies.dat"
users_path = "ml-1m/users.dat"

data = pd.read_csv(data_path, sep="::", names=["user", "item", "rating", "timestamp"], engine="python")
movies = pd.read_csv(movies_path, sep="::", names=["item", "title", "genres"], engine="python", encoding="ISO-8859-1")
users = pd.read_csv(users_path, sep="::", names=["user", "gender", "age", "occupation", "zip"], engine="python")

In [20]:
# reindex
user_mapping = {old:new for new, old in enumerate(users["user"].unique())}
item_mapping = {old:new for new, old in enumerate(movies["item"].unique())}
data["user"] = data["user"].map(user_mapping)
data["item"] = data["item"].map(item_mapping)
movies["item"] = movies["item"].map(item_mapping)
users["user"] = users["user"].map(user_mapping)

num_users = len(user_mapping)
num_items = len(item_mapping)

In [22]:
data['rating'].values.astype(np.float32)

array([5., 3., 3., ..., 5., 4., 4.], dtype=float32)

In [25]:
# define adjacency matrix
def build_adj_matrix(data, num_users, num_items):
    rows, cols = data['user'].values, data['item'].values
    interactions = data['rating'].values.astype(np.float32)
    adj_matrix = sp.coo_matrix((interactions, (rows, cols + num_users)), 
                               shape=(num_users+num_items, num_users+num_items))
    adj_matrix = adj_matrix + adj_matrix.T # make the graph undirected
    return adj_matrix

adj_matrix = build_adj_matrix(data=data, num_users=num_users, num_items=num_items)

def normalize_adj(adj):
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return d_mat_inv_sqrt @ adj @ d_mat_inv_sqrt

adj_matrix = normalize_adj(adj_matrix)
adj_matrix = torch.FloatTensor(adj_matrix.toarray())

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_17204/1830612108.py:14: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


In [26]:
# Handle user features
gender_map = {'M':0, 'F':1}
users['gender'] = users['gender'].map(gender_map)
user_features = torch.FloatTensor(users[['gender', 'age', 'occupation']].values)

# Handle item features  
genres_set = set('|'.join(movies['genres']).split('|'))
genre_mapping = {genre: i for i, genre in enumerate(genres_set)}
movies['genres_encoded'] = movies['genres'].apply(lambda x: [genre_mapping[g] for g in x.split('|')])
movie_features = torch.zeros((num_items, len(genres_set)))
for i, row in movies.iterrows():
    for g in row['genres_encoded']:
        movie_features[row['item'], g] = 1

In [ ]:
class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, n_layers, user_feat_dim, item_feat_dim):
        super(LightGCN, self).__init__()
        # define user and item embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        # project user and item features to embedding space
        self.user_feat_fc = nn.Linear(user_feat_dim, embedding_dim) 
        self.item_feat_fc = nn.Linear(item_feat_dim, embedding_dim)
        self.n_layers = n_layers
        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

    def forward(self, adj, user_features, item_features):
        user_feat_embed = self.user_feat_fc(user_features)
        item_feat_embed = self.item_feat_fc(item_features)
        all_embeddings = torch.cat([self.user_embedding.weight + user_feat_embed, self.item_embedding.weight + item_feat_embed], dim=0)
        embeddings = [all_embeddings]
        
        for _ in range(self.n_layers):
            all_embeddings = torch.mm(adj, all_embeddings)
            embeddings.append(all_embeddings)
        
        final_embedding = torch.mean(torch.stack(embeddings, dim=0), dim=0)
        user_final, item_final = torch.split(final_embedding, [num_users, num_items])
        return user_final, item_final

model = LightGCN(num_users, num_items, embedding_dim=64, n_layers=3, user_feat_dim=3, item_feat_dim=len(genres_set))
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

def train(model, data, adj, user_features, item_features, epochs=5):
    model.train()
    for epoch in range(epochs):
        user_embed, item_embed = model(adj, user_features, item_features)
        loss = 0
        for user, item, rating in zip(data['user'], data['item'], data['rating']):
            user_vec = user_embed[user]
            item_vec = item_embed[item]
            score = torch.sum(user_vec * item_vec)
            loss += criterion(score, torch.tensor(float(rating)))
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
        
train(model, data, adj_matrix, user_features, movie_features, epochs=5)